In [1]:
!pip install pyqt5


Defaulting to user installation because normal site-packages is not writeable
  Using cached PyQt5-5.15.11-cp38-abi3-win_amd64.whl.metadata (2.1 kB)
  Using cached PyQt5_Qt5-5.15.2-py3-none-win_amd64.whl.metadata (552 bytes)
Using cached PyQt5-5.15.11-cp38-abi3-win_amd64.whl (6.9 MB)
Using cached PyQt5_Qt5-5.15.2-py3-none-win_amd64.whl (50.1 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import cv2
print(cv2.__version__)
import PyQt5
print("PyQt5 installed successfully!")



4.11.0
PyQt5 installed successfully!


In [8]:
import sys
import numpy as np
import cv2
import tensorflow as tf
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QLabel
from PyQt5.QtGui import QPainter, QPen, QImage
from PyQt5.QtCore import Qt, QPoint

class DrawApp(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Draw a Number")
        self.setGeometry(100, 100, 400, 400)
        self.setStyleSheet("background-color: white;")

        self.image = QImage(400, 400, QImage.Format_RGB32)
        self.image.fill(Qt.white)

        self.clear_button = QPushButton("Clear", self)
        self.clear_button.setGeometry(150, 350, 100, 40)
        self.clear_button.clicked.connect(self.clear_canvas)

        self.predict_button = QPushButton("Predict", self)
        self.predict_button.setGeometry(260, 350, 100, 40)
        self.predict_button.clicked.connect(self.predict_digit)

        self.result_label = QLabel("Prediction: ", self)
        self.result_label.setGeometry(20, 360, 120, 30)

        self.model = tf.keras.models.load_model("model.h5")  # Load your trained model

        self.drawing = False
        self.last_point = QPoint()

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.drawing = True
            self.last_point = event.pos()

    def mouseMoveEvent(self, event):
        if self.drawing and event.buttons() & Qt.LeftButton:
            painter = QPainter(self.image)
            pen = QPen(Qt.black, 20, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin)
            painter.setPen(pen)
            painter.drawLine(self.last_point, event.pos())
            self.last_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.drawing = False

    def paintEvent(self, event):
        canvas = QPainter(self)
        canvas.drawImage(self.rect(), self.image, self.image.rect())

    def clear_canvas(self):
        self.image.fill(Qt.white)
        self.update()

    def preprocess_image(self):
        img = self.image.convertToFormat(QImage.Format_Grayscale8)
        buffer = img.bits().asarray(400 * 400)
        img_array = np.array(buffer, dtype=np.uint8).reshape(400, 400)

        img_resized = cv2.resize(img_array, (28, 28))
        img_resized = img_resized.astype("float32") / 255.0
        img_resized = np.expand_dims(img_resized, axis=(0, -1))  # Make it (1, 28, 28, 1)
        return img_resized

    def predict_digit(self):
        processed_img = self.preprocess_image()
        prediction = self.model.predict(processed_img)
        digit = np.argmax(prediction)
        self.result_label.setText(f"Prediction: {digit}")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = DrawApp()
    window.show()
    sys.exit(app.exec_())


RuntimeError: wrapped C/C++ object of type DrawApp has been deleted